---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

#### Option 2 ####
The second option is to design your own project that includes any of the algorithms we've discussed throughout the semester, or an algorithm that you're interested in learning that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** Brady Stark

---


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time
import random 

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [115]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4, nexp = 0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.nexp = nexp 

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # your code goes here...
        if move not in state.moves:
            return state
        row,col = move
        newState = cp.deepcopy(state)
        newState.utility = self.compute_utility(move,state)
        for i in range(self.nrow):
            if (row+1,col) in state.moves:
                row = row + 1
        newState.board[(row,col)] = state.to_move
        newState.moves.remove((row,col))
        newState.to_move = ('B' if state.to_move == 'R' else 'R')
        return newState
    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        # your code goes here...
        #print(move)
        row,col = move
        player = state.to_move
        tempBoard = cp.deepcopy(state.board)
        for i in range(self.nrow):
            if (row+1,col) in state.moves:
                row = row + 1
        tempBoard[(row,col)] = player
        #print(state.board)
        #Check Row
        rowCount = 0
        Left = 0
        Right = 0
        i = col + 1
        while i < self.ncol+1:
            if tempBoard.get((row,i))==player:
                Right = Right + 1
                i = i + 1
            else:
                i = self.ncol+2
        j = col-1
        while j > 0:
            if tempBoard.get((row,j))==player:
                Left = Left + 1
                j = j - 1
            else:
                j = 0
        rowCount = 1 + Left + Right
        #Check Col
        colCount = 0
        Up = 0
        Down = 0
        i = row - 1
        while i > 0:
            if tempBoard.get((i,col))==player:
                Up = Up + 1
                i = i - 1
            else:
                i = 0
        j = row + 1
        while j < self.nrow+1:
            if tempBoard.get((j,col))==player:
                Down = Down + 1
                j = j + 1
            else:
                j = self.nrow+2
        colCount = 1 + Up + Down
        #Check SW->NE Diagonal
        dia1 = 0
        UpR = 0
        DwL = 0
        i = col + 1
        j = row - 1
        while (i < self.ncol+1) & (j > 0):
            if tempBoard.get((j,i))==player:
                UpR = UpR + 1
                i = i + 1
                j = j - 1
            else:
                j = 0
        i = col - 1
        j = row + 1
        while (i > 0) & (j < self.nrow+1):
            if tempBoard.get((j,i))==player:
                DwL = DwL + 1
                i = i - 1
                j = j + 1
            else:
                i = 0
        dia1 = 1 + UpR + DwL
        #Check SE->NW Diagonal
        dia2 = 0
        UpL = 0
        DwR = 0
        i = col - 1
        j = row - 1
        while (i > 0) & (j > 0):
            if tempBoard.get((j,i))==player:
                UpL = UpL + 1
                i = i - 1
                j = j - 1
            else:
                j = 0
        i = col + 1
        j = row + 1
        while (i < self.ncol+1) & (j < self.nrow+1):
            if tempBoard.get((j,i))==player:
                DwR = DwR + 1
                i = i + 1
                j = j + 1
            else:
                j = self.nrow+2
        dia2 = 1 + UpL + DwR
        #for row in range(1, self.nrow + 1):
            #for col in range(1, self.ncol + 1):
                #print(tempBoard.get((row, col), '.'), end=' ')
            #print()
        if self.nwin in [rowCount,colCount,dia1,dia2]:
            #print("Connect 4")
            #print(move)
            #if self.nwin == rowCount:
                #print("In a row")
            #if self.nwin == colCount:
                #print("In a column")
            #if self.nwin == dia1:
                #print("SW->NE Diagonal")
            #if self.nwin == dia2:
                #print("Other Diagonal")
            if player == 'R':
                return 1
            if player == 'B':
                return -1
        else:
            return 0
        
    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        
        # your code goes here...   
        return state.utility!=0 or len(state.moves)==0
        
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        # your code goes here...
        return state.utility if player == 'R' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                #self.display()
                move = player(self)
                self.state = self.result(move, self.state)
                #self.display()
                if self.game_over(self.state):
                    #self.display()
                    return self.state.utility 
    def next_expend(self):
        return self.nexp
    def calcH(self,state):
        #self.display()
        player = state.to_move
        RHigh = 0
        BHigh = 0
        tempBoard = cp.deepcopy(state.board)
        values = []
        players = []
        for x in tempBoard:
            row,col = x
            checkPlayer = tempBoard[(row,col)]
            rowCount = 0
            Left = 0
            Right = 0
            i = col + 1
            flag = 0
            #Check Row
            while i < self.ncol+1:
                if tempBoard.get((row,i))==checkPlayer:
                    Right = Right + 1
                    i = i + 1
                else:
                    if tempBoard.get((row,i)):
                        flag=flag+1
                    i = self.ncol+2
            j = col-1
            if j==0:
                Left = -1
            while j > 0:
                if tempBoard.get((row,j))==checkPlayer:
                    Left = Left + 1
                    j = j - 1
                else:
                    if tempBoard.get((row,j)):
                        flag=flag+1
                    j = 0
            rowCount = 1 + Left + Right
            if flag == 2:
                rowCount = 0
            #Col Count
            flag = 0
            colCount = 0
            Up = 0
            Down = 0
            i = row - 1
            if i==0:
                Up = -1
            while i > 0:
                if tempBoard.get((i,col))==checkPlayer:
                    Up = Up + 1
                    i = i - 1
                else:
                    if tempBoard.get((i,col)):
                        flag=flag+1
                    i = 0
            j = row + 1
            while j < self.nrow+1:
                if tempBoard.get((j,col))==checkPlayer:
                    Down = Down + 1
                    j = j + 1
                else:
                    if tempBoard.get((j,col)):
                        flag=flag+1
                    j = self.nrow+2
            colCount = 1 + Up + Down
            if flag == 2:
                colCount = 0
            #Check SW->NE Diagonal
            flag = 0
            dia1 = 0
            UpR = 0
            DwL = 0
            i = col + 1
            j = row - 1
            while (i < self.ncol+1) & (j > 0):
                if tempBoard.get((j,i))==checkPlayer:
                    UpR = UpR + 1
                    i = i + 1
                    j = j - 1
                else:
                    if tempBoard.get((j,i)):
                        flag = flag + 1
                    j = 0
            i = col - 1
            j = row + 1
            while (i > 0) & (j < self.nrow+1):
                if tempBoard.get((j,i))==checkPlayer:
                    DwL = DwL + 1
                    i = i - 1
                    j = j + 1
                else:
                    if tempBoard.get((j,i)):
                        flag = flag + 1
                    i = 0
            dia1 = 1 + UpR + DwL
            if flag == 2:
                dia1 = 0
            #Check SE->NW Diagonal
            flag = 0
            dia2 = 0
            UpL = 0
            DwR = 0
            i = col - 1
            j = row - 1
            while (i > 0) & (j > 0):
                if tempBoard.get((j,i))==checkPlayer:
                    UpL = UpL + 1
                    i = i - 1
                    j = j - 1
                else:
                    if tempBoard.get((j,i)):
                        flag = flag + 1
                    j = 0
            i = col + 1
            j = row + 1
            while (i < self.ncol+1) & (j < self.nrow+1):
                if tempBoard.get((j,i))==checkPlayer:
                    DwR = DwR + 1
                    i = i + 1
                    j = j + 1
                else:
                    if tempBoard.get((j,i)):
                        flag = flag + 1
                    j = self.nrow+2
            dia2 = 1 + UpL + DwR
            if flag == 2:
                dia2 = 0
            scores = [rowCount,colCount,dia1,dia2]
            values.append(max(scores))
            players.append(checkPlayer)
        mValue = max(values)
        rCount = 0
        bCount = 0
        for i in range(len(values)):
            if values[i] == mValue:
                if players[i] == 'R':
                    rCount = rCount + 1
                if players[i] == 'B':
                    bCount = bCount + 1
        if rCount > 0:
            rCount = rCount * (mValue * mValue)
        if bCount > 0:
            bCount = bCount * (mValue * mValue)
        return(rCount - bCount)

In [109]:
test = ConnectFour()
test.state.board[(5,2)] = 'R'
test.state.board[(6,2)] = 'R'
test.state.board[(6,3)] = 'B'
test.state.board[(6,4)] = 'B'
test.state.board[(4,2)] = 'R'
print(test.state.board)
test.calcH(test.state)

{(5, 2): 'R', (6, 2): 'R', (6, 3): 'B', (6, 4): 'B', (4, 2): 'R'}
. . . . . . . 
. . . . . . . 
. . . . . . . 
. R . . . . . 
. R . . . . . 
. R B B . . . 


27

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [113]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    # your code goes here...
    return random.choice(game.state.moves)


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [6]:
rWins = 0
bWins = 0
draws = 0
for i in range(1000):
    game = ConnectFour()
    run = game.play_game(random_player,random_player)
    if run == 1:
        rWins += 1
    elif run == -1:
        bWins += 1
    else:
        draws +=1
print(rWins,bWins,draws)

567 432 1


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [124]:
rWins = 0
bWins = 0
draws = 0
for i in range(1000):
    game = ConnectFour(10,10,6)
    run = game.play_game(random_player,random_player)
    #print(run)
    if run == 1:
        rWins +=1
    elif run == -1:
        bWins +=1
    else:
        draws +=1
print(rWins,bWins,draws)

492 391 117


It seems as though increasing the size of the board makes the game more fair to both players as their number of draws are higher. This could be because with a larger board there is less blocking which means that going first is not as advantageous. 

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [120]:
def alphabeta_player(game):
    player = game.state.to_move
    def max_value(state,alpha,beta):
        if game.game_over(state):
            return game.utility(state,player)
        v = -np.inf
        #print(state.board)
        for a in state.moves:
            game.nexp=game.nexp+1
            v = max(v,min_value(game.result(a,state),alpha,beta))
            if v >= beta:
                return v
            alpha = max(alpha,v)
        return v
    def min_value(state,alpha,beta):
        if game.game_over(state):
            return game.utility(state,player)
        v = np.inf
        #print(state.moves)
        for a in state.moves:
            game.nexp=game.nexp+1
            v = min(v,max_value(game.result(a,state),alpha,beta))
            if v <= alpha:
                return v
            beta = min(beta,v)
        return v
    bestScore = -np.inf
    beta = np.inf
    bestAction = None
    for a in game.state.moves:
        v = min_value(game.result(a,game.state),bestScore,beta)
        game.nexp=game.nexp+1
        if v > bestScore:
            bestScore = v
            bestAction = a
    return bestAction

In [17]:
game = ConnectFour(3,4,3)
run = game.play_game(alphabeta_player,random_player)

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [22]:
rWins = 0
bWins = 0
draws = 0
for i in range(10):
    game = ConnectFour(3,4,3)
    run = game.play_game(alphabeta_player,random_player)
    #print(run)
    if run == 1:
        rWins +=1
    elif run == -1:
        bWins +=1
    else:
        draws +=1
print(rWins,bWins,draws)

10 0 0


In [30]:
rWins = 0
bWins = 0
draws = 0
for i in range(10):
    game = ConnectFour(3,4,3)
    run = game.play_game(alphabeta_player,alphabeta_player)
    #print(run)
    if run == 1:
        rWins +=1
    elif run == -1:
        bWins +=1
    else:
        draws +=1
print(rWins,bWins,draws)

10 0 0


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [5]:
# Your code here.
def minimax_player(game):
    player = game.state.to_move
    def max_value(state,alpha,beta):
        if game.game_over(state):
            return game.utility(state,player)
        v = -np.inf
        #print(state.board)
        for a in state.moves:
            game.nexp=game.nexp+1
            v = max(v,min_value(game.result(a,state),alpha,beta))
            alpha = max(alpha,v)
        return v
    def min_value(state,alpha,beta):
        if game.game_over(state):
            return game.utility(state,player)
        v = np.inf
        for a in state.moves:
            game.nexp=game.nexp+1
            v = min(v,max_value(game.result(a,state),alpha,beta))
            beta = min(beta,v)
        return v
    bestScore = -np.inf
    beta = np.inf
    bestAction = None
    for a in game.state.moves:
        v = min_value(game.result(a,game.state),bestScore,beta)
        game.nexp=game.nexp+1
        if v > bestScore:
            bestScore = v
            bestAction = a
    return bestAction

In [3]:
#game = ConnectFour(3,3,3)
#game.play_game(alphabeta_player,alphabeta_player)
#prune = game.next_expend()

In [4]:
#game2 = ConnectFour(3,3,3)
#game2.play_game(minimax_player,minimax_player)
#notPrune = game2.next_expend()

In [ ]:
#print("Saving:", prune/notPrune)

I was not able to get this to work I think it is because of the runtime but there may be something wrong with my code. However, the number of nodes expanded without pruning would be around 50 times larger than that with pruning.

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


In [128]:
def aStar(game):
    player = game.state.to_move
    scores = []
    bestAction = None
    for a in game.state.moves:
        if game.game_over(game.result(a,game.state)):
            return a
        v = game.calcH(game.result(a,game.state))
        scores.append(v)
    if player == 'R':
        mIndex = scores.index(max(scores))
        bestAction = game.state.moves[mIndex]
    if player == 'B':
        mIndex = scores.index(min(scores))
        bestAction = game.state.moves[mIndex]
    return bestAction

In [129]:
rWins = 0
bWins = 0
draws = 0
for i in range(10):
    game = ConnectFour(3,4,3)
    run = game.play_game(aStar,random_player)
    #print(run)
    if run == 1:
        rWins +=1
    elif run == -1:
        bWins +=1
    else:
        draws +=1
print(rWins,bWins,draws)

10 0 0


In [130]:
rWins = 0
bWins = 0
draws = 0
for i in range(10):
    game = ConnectFour(3,4,3)
    run = game.play_game(aStar,alphabeta_player)
    #print(run)
    if run == 1:
        rWins +=1
    elif run == -1:
        bWins +=1
    else:
        draws +=1
print(rWins,bWins,draws)

10 0 0


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

I designed my own heuristic function. What I did was similar to chess in the way that I calculated a score based on the state of the board. This score had to be representitive of which color had the advantage in the given postiton. So what I did was I went through the board and found the longest unbroken chains. From there I squared the size of the chain because I felt the size was the most important and I multiplied that by the number of chains of that length the color has. I took into account if both sides had a chain of equal length which is why I multiplied it by the number of chains. Finally, I subtracted the red score and the black score. This gave me a number that was accurate to show how close either side is to winning.

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

It seems as though the A* algorithm is a lot better than the random player and the AB pruning player. However, I am not sure how accurate the data is because of the runtime problems that have been happening during this project. It seems as though with the current board size the first player wins almost everytime as long as they have some sort of strategy. 

However, in theory I believe that the AB pruning player would beat out the A* player on a larger board because it can look further down the line at the end game whereas the A* player only looks straight forward but with more accuracy. If you combined the two I would imagine you get something a lot closer to a real bot that doesn't just look at the terminal states for utility but the state of the board. 

If I were to go back and improve the heurisitc I made I would start by fixing some logic when checking to see diagonal chains. The algorithm does not take into account if the chain can be completed in the near future, for example if there is a 3 in a row diagonal but no coins under the 4th slot then the 3 in a row is basically worthless until there is enough coins to get to the slot. 